In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import pandas_profiling as pp#data visualization


import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import boxcox,skew,norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# will do 76.9% reduction im memory usage
def reduce_mem_usage(df, verbose=True):#from santander ml comp
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
train= reduce_mem_usage(pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv'))
test= reduce_mem_usage(pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv'))
print("Shape of train set: ",train.shape)
print("Shape of test set: ",test.shape)
sample = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
train.drop(['id'],axis=1,inplace=True)

In [ ]:
train.isna().sum().sum()

In [ ]:
test.isna().sum().sum()

In [ ]:
gc.collect()

In [ ]:
#x,y 
x = train.drop(['target'],axis=1)
y =train.target

In [ ]:
#scaling by max 
x = x / x.max()
#scaling test
test =test/test.max()

In [ ]:
del train
gc.collect()

In [ ]:
#Spliting data in to train and test for model training
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)
gc.collect()

In [ ]:

##Import libraries for classification model 
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn import model_selection

In [ ]:
clf = lgb.LGBMClassifier()
rf_model =RandomForestClassifier(criterion='entropy',random_state=1,max_depth=10,n_estimators=21)
lr = LogisticRegression(solver='liblinear')


In [ ]:
rf_model.fit(x_train,y_train)
gc.collect()
clf.fit(x_train,y_train)
gc.collect()
lr.fit(x_train,y_train)

In [ ]:
gc.collect()

### Predict validation data and test data and use the predictions as new feature

In [ ]:
##validation
val_pred_rf = rf_model.predict_proba(x_test)
val_pred_lgb = clf.predict_proba(x_test)
val_pred_lr = lr.predict_proba(x_test)


In [ ]:
val_pred_rf = [x[1] for x in val_pred_rf]
val_pred_lgb = [x[1] for x in val_pred_lgb]
val_pred_lr = [x[1] for x in val_pred_lr]
gc.collect()

In [ ]:
#droping id from test
test =test.drop(['id'],axis=1)

In [ ]:
##getting predict probs
predictions_rf = rf_model.predict_proba(test)
predictions_lgb = clf.predict_proba(test)
predictions_lr= lr.predict_proba(test)

In [ ]:
predictions_rf = [x[1] for x in predictions_rf]
predictions_lgb = [x[1] for x in predictions_lgb]
predictions_lr = [x[1] for x in predictions_lr]

In [ ]:
val_pred_rf = pd.DataFrame(val_pred_rf,columns=['rf'])
val_pred_lgb = pd.DataFrame(val_pred_lgb,columns=['lgb'])
val_pred_lr = pd.DataFrame(val_pred_lr,columns=['lr'])

predictions_rf = pd.DataFrame(predictions_rf,columns=['rf'])
predictions_lgb = pd.DataFrame(predictions_lgb,columns=['lgb'])
predictions_lr = pd.DataFrame(predictions_lr,columns=['lr'])

In [ ]:
# Adding predicted values as new feature
x_test['rf'] = val_pred_rf['rf'].values
x_test['lgb'] = val_pred_lgb['lgb'].values
x_test['lr'] = val_pred_lr['lr'].values


In [ ]:
x_test.shape

In [ ]:
# Adding predicted values as new feature in test data also
test['rf'] = predictions_rf['rf'].values
test['lgb'] = predictions_lgb['lgb'].values
test['lr'] = predictions_lr['lr'].values

In [ ]:
#new  estimation
model = LogisticRegression(solver='liblinear')
model.fit(x_test,y_test)
model.score(x_test,y_test)

In [ ]:
hold_out_pred = model.predict_proba(test)
hold_out_pred = [x[1] for x in hold_out_pred]
sample['target'] = hold_out_pred

In [ ]:
#submission and see the result for the base M
sample.to_csv("submission.csv", index=False)

In [ ]:
# Will do the Auto EDA
# pp.ProfileReport(train)

In [ ]:
#continue